In [18]:
#imports
import os, shutil, fnmatch, tkinter
from tkinter.filedialog import askdirectory
from pathlib import Path
from woundcompute import image_analysis as ia
import time


In [19]:
#prompt user for the file directory. Will open as a popup window named "tk"
tk_root = tkinter.Tk()
print("Please open the directory that contains your .nd file")
path_input = askdirectory(title='Select Input Folder')  # shows dialog box and return the path
print(path_input)
tk_root.destroy()
path_output = path_input + '/Sorted'

if os.path.exists(path_output):
    path_out_new = input('Enter new output folder name')
    path_output = path_input + '/' + path_out_new
    os.makedirs(path_output)
#create a new  output directory
else:
    os.makedirs(path_output)

Please open the directory that contains your .nd file
E:/Code_processed_eclipse_data/20230105_NHDFneo_P5_Irrad-ECM


In [20]:
#find the experiment base names based on the .nd files in the folder
basename_list = []
for file in os.listdir(path_input):
    if file.endswith('.nd'):
        name, ext = os.path.splitext(file)
        basename_list.append(name)
        #create folders for each expt
        os.makedirs(path_output + '/' + name)
print(basename_list)

['tissue_ai', 'tissue_ai1']


In [21]:
#copy files into folders grouped by expt base names
stage_pos_nd = []
for basename in basename_list:

    #copy .nd file into respective basename folder
    shutil.copy(path_input + '/' + basename + '.nd', path_output + '/' + basename + '/' + basename + '.nd')
    with open(path_output + '/' + basename + '/' + basename + '.nd', 'r') as nd_file:
        for l_no, line in enumerate(nd_file):
            if '"NStagePositions"' in line:
                stage_pos_nd.append([int(s) for s in line.split() if s.isdigit()][-1])
                break
    #copy TIF files to respective basename folders
    for file in os.listdir(path_input):
        if file.startswith(basename + '_'):
            shutil.copy(path_input + '/' + file, path_output + '/' + basename + '/' + file)



In [22]:
#delete thumbnail files in folder
for basename in basename_list:
    for file in os.listdir(path_output + '/' + basename):
        if fnmatch.fnmatch(file, '*thumb*'):
            # Delete the thumbnail files - with exception handling
            try:
                os.remove(path_output + '/' + basename + '/' + file)
            except OSError as e:
                # If it fails, inform the user.
                print('Error: %s - %s.' % (e.filename, e.strerror))#%%


In [25]:
import yaml

#Information to be stored in the yaml file
yaml_input_file = {
'version' : 1.0,
'segment_brightfield' : False,
'seg_bf_version' : 1,
'seg_bf_visualize' : False,
'segment_fluorescent' : False,
'seg_fl_version' : 1,
'seg_fl_visualize' : False,
'segment_ph1' : True,
'seg_ph1_version' : 2,
'seg_ph1_visualize' : True,
'track_brightfield' : False,
'track_bf_version' : 1,
'track_bf_visualize' : False,
'track_ph1' : False,
'track_ph1_version' : 1,
'track_ph1_visualize' : False,
'bf_seg_with_fl_seg_visualize' : False,
'bf_track_with_fl_seg_visualize' : False,
'ph1_seg_with_fl_seg_visualize' : False,
'ph1_track_with_fl_seg_visualize' : False,
}

with open(r'E:\data\store_file.yaml', 'w') as file:
    documents = yaml.dump(dict_file, file)


version : 1.0
segment_brightfield : False
seg_bf_version : 1
seg_bf_visualize : False
segment_fluorescent : False
seg_fl_version : 1
seg_fl_visualize : False
segment_ph1 : True
seg_ph1_version : 2
seg_ph1_visualize : True
track_brightfield : False
track_bf_version : 1
track_bf_visualize : False
track_ph1 : False
track_ph1_version : 1
track_ph1_visualize : False
bf_seg_with_fl_seg_visualize : False
bf_track_with_fl_seg_visualize : False
ph1_seg_with_fl_seg_visualize : False
ph1_track_with_fl_seg_visualize : False


In [23]:
#sort images into stage position folders
for index,basename in enumerate(basename_list):

    for file in os.listdir(path_output+'/'+basename):
        for stage_pos in range(1,stage_pos_nd[index]+1):

            #check if file belongs to current stage_pos group
            if fnmatch.fnmatch(file, '*s%i' % stage_pos+'_*'):

                #rename file for correct sort syntax
                file_timepoint = file.split('_t', 1)[-1].split('.', 1)[0]

                for num in range(3-len(file_timepoint)):
                    file_timepoint = '0'+file_timepoint
                filename_new = 's%i' % stage_pos+'_t'+file_timepoint+'.TIF'

                #check if stage position directory already exists and move file into this folder if it does
                if os.path.exists(path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images'):
                    try:
                        shutil.move(path_output+'/'+basename+'/'+file, path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images/'+filename_new)
                    except OSError as e:
                        # If it fails, inform the user.
                        print('Error: %s - %s.' % (e.filename, e.strerror))

                else:
                    try:
                        os.makedirs(path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images')
                        shutil.copy('G:/My Drive/Boston University/Chen Lab/Code/Anish/wc_dataset_ph1.yaml', path_output+'/'+basename+'/'+'s%i' % stage_pos+'/wc_dataset_ph1.yaml')
                        shutil.move(path_output+'/'+basename+'/'+file, path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images/'+filename_new)

                    except OSError as e:
                        # If it fails, inform the user.
                        print('Error: %s - %s.' % (e.filename, e.strerror))
            elif fnmatch.fnmatch(file, '*s%i' % stage_pos+'.TIF'):
                if os.path.exists(path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images'):
                    try:
                        shutil.move(path_output+'/'+basename+'/'+file, path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images/'+file)
                    except OSError as e:
                        # If it fails, inform the user.
                        print('Error: %s - %s.' % (e.filename, e.strerror))
                # if the directory does not already exist, create directory and move file into it
                else:
                    try:
                        os.makedirs(path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images')
                        shutil.copy('G:/My Drive/Boston University/Chen Lab/Code/Anish/wc_dataset_ph1.yaml', path_output+'/'+basename+'/'+'s%i' % stage_pos+'/wc_dataset_ph1.yaml')
                        shutil.move(path_output+'/'+basename+'/'+file, path_output+'/'+basename+'/'+'s%i' % stage_pos+'/ph1_images/'+file)

                    except OSError as e:
                        # If it fails, inform the user.
                        print('Error: %s - %s.' % (e.filename, e.strerror))

In [24]:
for index, basename in enumerate(basename_list):

    for stage_pos in range(1, stage_pos_nd[index]+1):
        input_location = Path(path_output+'/'+basename+'/'+'s%i' % stage_pos)
        try:
            time_all, action_all = ia.run_all(input_location)
            print("tissue number:", stage_pos, "time:", time.time())
        except Exception as ex:
            time_all.append(time.time())
            print("tissue number:", stage_pos, "time:", time.time())
            print("---------ERROR OF SOME DESCRIPTION HAS HAPPENED-------")
            print(ex)
            print("------------------------------------------------------")

tissue number: 1 time: 1680541614.3649378


KeyboardInterrupt: 